In [ ]:
import pandas as pd
from datetime import datetime
import trdb2py

isStaticImg = False
width = 960
height = 768

pd.options.display.max_columns = None
pd.options.display.max_rows = None

trdb2cfg = trdb2py.loadConfig('./trdb2.yaml')

In [ ]:
lstassetcode = ['jqdata.000016_XSHG|1d', 
                'jqdata.000300_XSHG|1d', 
                'jqdata.000905_XSHG|1d', 
                'jqdata.000032_XSHG|1d', 
                'jqdata.000033_XSHG|1d', 
                'jqdata.000034_XSHG|1d', 
                'jqdata.000036_XSHG|1d', 
                'jqdata.000037_XSHG|1d', 
                'jqdata.000038_XSHG|1d', 
                'jqdata.000039_XSHG|1d', 
                'jqdata.000040_XSHG|1d', 
                'jqdata.000043_XSHG|1d', 
                'jqdata.000044_XSHG|1d', 
                'jqdata.000045_XSHG|1d', 
                'jqdata.000046_XSHG|1d']
lstassetname = ['上证50', 
                '沪深300', 
                '中证500', 
                '上证能源', 
                '上证材料', 
                '上证工业', 
                '上证消费', 
                '上证医药', 
                '上证金融', 
                '上证信息', 
                '上证电信', 
                '超大盘', 
                '上证中盘', 
                '上证小盘', 
                '上证中小']

# 起始时间，0表示从最开始算起
# tsStart = 0
tsStart = int(trdb2py.str2timestamp('2020-10-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1

ret = trdb2py.calcAssetsSimilarity(trdb2cfg, 'jqdata.000300_XSHG|1d', 'jqdata.000032_XSHG|1d', tsStart, tsEnd)

ret

In [ ]:
ret = trdb2py.calcAssetsSimilarity2(trdb2cfg, lstassetname, lstassetcode, tsStart, tsEnd)

df = {'name': []}
for name in lstassetname:
    df[name] = []
    df['name'].append(name)
    
for i in range(len(lstassetname)):
    for j in range(len(lstassetname)):
        df[lstassetname[i]].append(ret[i][j])

df1 = pd.DataFrame(df)
df1

In [ ]:
lstparams = []

for ai in range(0, len(lstassetcode)):
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(lstassetcode[ai]),         
    )

    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='buyandhold',
    )

    paramsbuy = trdb2py.trading2_pb2.BuyParams(
        perHandMoney=1,
    )

    paramsinit = trdb2py.trading2_pb2.InitParams(
        money=10000,
    )

    s0.buy.extend([buy0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsInit.CopyFrom(paramsinit)        
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(lstassetcode[ai])],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title=lstassetname[ai],
    )
    
    lstparams.append(p0)

lstpnl1 = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstpnl1, toImg=isStaticImg, width=width, height=height)